In [1]:
import pymongo
import folium
import pandas as pd
import json
from pandas.io.json import json_normalize

In [2]:
cliente=pymongo.MongoClient()
database=cliente.companies
db=database.companies

In [3]:
look = pd.DataFrame(db.find({'deadpooled_year': None}, {'name':1, 'category_code':1, 'founded_year':1, 'ipo':1, '_id':0}))
look.head()

,name,category_code,founded_year,ipo
0,Digg,news,2004.0,None
1,Facebook,social,2004.0,"{'valuation_amount': 104000000000, 'valuation_..."
2,Postini,web,1999.0,None
3,Geni,web,2006.0,None
4,Flektor,games_video,NaN,None


In [4]:
look.groupby('category_code').count()

,name,founded_year,ipo
category_code,,,
advertising,892,783,13
analytics,66,66,0
automotive,8,6,2
biotech,355,223,26
cleantech,281,193,17
consulting,630,525,9
design,4,4,0
ecommerce,646,542,5
education,36,33,2


In [5]:
look.founded_year.max()

2013.0

In [6]:
look.shape

(17872, 4)

# Requerimientos del cliente
#### 1. Quiere ubicar empresa de Cleantech
#### 2. Cerca de empresas jóvenes
#### 3. Prioritariamente cerca de empresas de giro social, salud y noticias

In [7]:
selection = pd.DataFrame(db.find({'deadpooled': None, 
                                  '$or': [{'category_code':'cleantech'},
                                          {'category_code':'health'}, 
                                          {'category_code':'social'}, 
                                          {'category_code':'news'}],
                                  'founded_year':{'$gt':2003},
                                  'offices.0.country_code':'USA'},
                                {'_id':0, 'name':1, 'offices':1, 'category_code':1, 'founded_year':1}))
selection.head()

,name,category_code,founded_year,offices
0,Digg,news,2004,"[{'description': None, 'address1': '135 Missis..."
1,Facebook,social,2004,"[{'description': 'Headquarters', 'address1': '..."
2,Twitter,social,2006,"[{'description': '', 'address1': '1355 Market ..."
3,Scribd,news,2007,"[{'description': 'HQ', 'address1': '539 Bryant..."
4,MeetMoi,social,2007,"[{'description': None, 'address1': '', 'addres..."


In [8]:
selection['offices'].iloc[0]

[{'description': None,
  'address1': '135 Mississippi St',
  'address2': '',
  'zip_code': '94107',
  'city': 'San Francisco',
  'state_code': 'CA',
  'country_code': 'USA',
  'latitude': 37.764726,
  'longitude': -122.394523}]

In [10]:
selection.columns

Index(['name', 'category_code', 'founded_year', 'offices'], dtype='object')

In [11]:
selection.offices = selection.offices.apply(lambda x: x[0])

In [12]:
df = pd.concat([json_normalize(selection.offices), selection[['name', 'founded_year', 'category_code']]], axis=1)
df.shape

(139, 12)

In [13]:
df.drop(df[df.latitude.isnull()].index, inplace=True)

In [14]:
df.shape

(111, 12)

In [15]:
df.drop_duplicates(inplace=True)

In [16]:
df.shape

(107, 12)

In [17]:
df.head()

,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,name,founded_year,category_code
0,None,135 Mississippi St,,94107,San Francisco,CA,USA,37.764726,-122.394523,Digg,2004,news
1,Headquarters,1601 Willow Road,,94025,Menlo Park,CA,USA,37.416050,-122.151801,Facebook,2004,social
2,,1355 Market St.,,94103,San Francisco,CA,USA,37.776805,-122.416924,Twitter,2006,social
3,HQ,539 Bryant Street,,94107,San Francisco,CA,USA,37.789634,-122.404052,Scribd,2007,news
4,None,,,,New York City,NY,USA,40.757929,-73.985506,MeetMoi,2007,social


In [18]:
df.reset_index(inplace=True)

## Análisis con Folium

In [19]:
mapa=folium.Map(location=[39.67348, -100.496137], tiles='openstreetmap',zoom_start=4.2)

In [20]:
for index in df.index:
    place=df.loc[index]
    popupText = '<i>' + place['name'] + '</i>'
    folium.Marker([place.latitude, place.longitude], popup=popupText, tooltip=place.category_code).add_to(mapa)
mapa

### Después de ver el resultado, el cliente está decidido a ubicarse en el lugar más céntrico a todas las demás empresas de la selección, a la ciudad más cercana.

In [40]:
lat = df.latitude.mean()
long = df.longitude.mean()

In [50]:
mapa=folium.Map(location=[39.67348, -100.496137], tiles='openstreetmap',zoom_start=4.2)
folium.Circle([lat, long], radius=250000, icon=folium.Icon()).add_to(mapa)
mapa

In [26]:
air = pd.read_csv('airquality.csv')
air

,Rank,Air Quality Index ▼,State / Population
0,1,51.2,"Utah / 2,858,111"
1,2,48.2,"Ohio / 11,560,380"
2,2,48.2,"Georgia / 9,907,756"
3,4,47.6,"West Virginia / 1,853,881"
4,5,47.5,"Indiana / 6,542,411"
5,5,47.5,"Tennessee / 6,451,365"
6,7,47.1,"Colorado / 5,197,580"
7,8,47.0,"Maryland / 5,887,776"
8,9,46.8,"District of Columbia / 633,736"
9,10,46.6,"Alabama / 4,817,678"


In [33]:
air['State'] = air['State / Population'].apply(lambda x: x.split('/')[0].strip())

In [35]:
air['Population'] = air['State / Population'].apply(lambda x: x.split('/')[1].strip())

In [36]:
air.head()

,Rank,Air Quality Index ▼,State / Population,State,Population
0,1,51.2,"Utah / 2,858,111",Utah,"2,858,111"
1,2,48.2,"Ohio / 11,560,380",Ohio,"11,560,380"
2,2,48.2,"Georgia / 9,907,756",Georgia,"9,907,756"
3,4,47.6,"West Virginia / 1,853,881",West Virginia,"1,853,881"
4,5,47.5,"Indiana / 6,542,411",Indiana,"6,542,411"
